<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-2">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-3">Main</a></span><ul class="toc-item"><li><span><a href="#Load-all-pubrep-files-and-make-a-metadata" data-toc-modified-id="Load-all-pubrep-files-and-make-a-metadata-3.1">Load all pubrep files and make a metadata</a></span></li><li><span><a href="#add-study-name" data-toc-modified-id="add-study-name-3.2">add study name</a></span></li><li><span><a href="#Extract-patient-id-from-samplename" data-toc-modified-id="Extract-patient-id-from-samplename-3.3">Extract patient id from samplename</a></span></li><li><span><a href="#Extract-sampletype" data-toc-modified-id="Extract-sampletype-3.4">Extract sampletype</a></span></li></ul></li></ul></div>

# Dependencies

In [1]:
library(data.table)

library(ggplot2)

Warning message:
“package ‘data.table’ was built under R version 3.5.2”Warning message:
“package ‘ggplot2’ was built under R version 3.5.2”

# Paths

In [50]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/Jabado/TCR_HSJ/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/Jabado/TCR_HSJ/Plots/"

# Main

## Load all pubrep files and make a metadata

In [163]:
filelist <- list.files(datapath,
                     recursive = TRUE,
                     pattern = "pubrep")

In [164]:
filelist <- filelist[grepl("common_clonotypes", filelist)]

In [165]:
filelist

[1] "common_clonotypes/pubrep_ims_batch1to44.csv" 
 [2] "common_clonotypes/pubrep_ims_lines.csv"      
 [3] "common_clonotypes/pubrep_inspire_cfDNA.csv"  
 [4] "common_clonotypes/pubrep_inspire_genomic.csv"
 [5] "common_clonotypes/pubrep_INTERCEPT.csv"      
 [6] "common_clonotypes/pubrep_LFS.csv"            
 [7] "common_clonotypes/pubrep_lymphotrack.csv"    
 [8] "common_clonotypes/pubrep_NPCcaptured.csv"    
 [9] "common_clonotypes/pubrep_TCLlines.csv"       
[10] "common_clonotypes/pubrep_TCR_HSJ.csv"        
[11] "common_clonotypes/pubrep_TLML_cfDNA.csv"     
[12] "common_clonotypes/pubrep_TLMLcDNAgenomic.csv"

In [166]:
all.files <- lapply(filelist, function(f){ read.csv(paste0(datapath,f), stringsAsFactors = F,
                                                   header = F, nrow = 1)})

In [167]:
allfilenames <- unlist(all.files)

In [190]:
allfilenames <- allfilenames[allfilenames != "CDR3.aa" &
                             allfilenames != "Samples"]

In [191]:
length(allfilenames)

[1] 518

In [192]:
allfilenames_df <- as.data.frame(allfilenames)

In [193]:
allfilenames_df$Study <- NA
allfilenames_df$Patient <- NA
allfilenames_df$Samplename <- NA
allfilenames_df$Sample_type <- NA

In [194]:
colnames(allfilenames_df)[colnames(allfilenames_df) == "allfilenames"] <- "filename"

In [195]:
head(allfilenames_df)

filename,Study,Patient,Samplename,Sample_type
<fct>,<lgl>,<lgl>,<lgl>,<lgl>
IMS-001_MoST-K006_T_DNA,NA,NA,NA,NA
IMS-002_MoST-K011_T_DNA,NA,NA,NA,NA
IMS-003_MoST-K014_T_DNA,NA,NA,NA,NA
IMS-004_MoST-K001_T_DNA,NA,NA,NA,NA
IMS-005_MoST-K016_T_DNA,NA,NA,NA,NA
IMS-006_MoST-K017_T_DNA,NA,NA,NA,NA


In [196]:
allfilenames_df$filename <- as.character(allfilenames_df$filename)

In [197]:
allfilenames_df$Samplename <- gsub("CLONES_TRB_", "", allfilenames_df$filename)
allfilenames_df$Samplename <- gsub("CLONES_TRB", "", allfilenames_df$Samplename)

## add study name

In [198]:
HSJ <- c('23-Blood', '23-Tum', '50-Tum', '51-Tum', '23-Tum_2000000', 
         '50-Tum_2000000', '51-Tum_2000000', '023-CSF', '051-CSF', '051R-CSF',
         '19-CSF', '41-CSF', '67-CSF', '76-Tum', '77-Tum', '19-CSF_2000000',
         '23-CSF_2000000', '41-CSF_2000000', '51-CSF_2000000', '51R-CSF_2000000',
         '67-CSF_2000000', '76-Tum_2000000', '77-Tum_2000000', '23-Plasma',
         '50-CSF', '67-Tum', '68-Plasma', '68-Tum', '76-CSF', '76-Plasma',
         '77-CSF', '77-Plasma', '50-CSF_2000000', '67-Tum_2000000', 
         '68-Plasma_2000000', '76-CSF_2000000','76-Plasma_2000000')

In [213]:
allfilenames_df$Study[grepl("IMS",allfilenames_df$filename)] <- "IMS"
allfilenames_df$Study[grepl("INSPIRE",allfilenames_df$filename)] <- "INSPIRE"
allfilenames_df$Study[grepl("CHP",allfilenames_df$filename)] <- "INTERCEPT"
allfilenames_df$Study[grepl("LFS",allfilenames_df$filename)] <- "LFS"
allfilenames_df$Study[grepl("Lymphotrack",allfilenames_df$filename)] <- "Lymphotrack"
allfilenames_df$Study[grepl("NPC",allfilenames_df$filename)] <- "NPC"
allfilenames_df$Study[grepl("TLML",allfilenames_df$filename)] <- "TLML"
allfilenames_df$Study[grepl("TLDC",allfilenames_df$filename)] <- "TLDC"
allfilenames_df$Study[allfilenames_df$Samplename %in% HSJ] <- "HSJ"

## Extract patient id from samplename

In [200]:
allfilenames_df$Patient[which(allfilenames_df$Study == "IMS")] <- substr(allfilenames_df$filename[which(allfilenames_df$Study == "IMS")], 0,7)
allfilenames_df$Patient[which(allfilenames_df$Study == "INSPIRE")] <- substr(allfilenames_df$Samplename[which(allfilenames_df$Study == "INSPIRE")], 9,17)
allfilenames_df$Patient[which(allfilenames_df$Study == "INTERCEPT")] <- substr(allfilenames_df$filename[which(allfilenames_df$Study == "INTERCEPT")], 0,7)
allfilenames_df$Patient[which(allfilenames_df$Study == "LFS")] <- substr(allfilenames_df$Samplename[which(allfilenames_df$Study == "LFS")], 0,12)
allfilenames_df$Patient[which(allfilenames_df$Study == "NPC")] <- substr(allfilenames_df$Samplename[which(allfilenames_df$Study == "NPC")], 20,32)

In [201]:
allfilenames_df$Patient[which(allfilenames_df$Study == "HSJ")&
                       grepl("B23", allfilenames_df$filename)] <- "HSJ23"
allfilenames_df$Patient[which(allfilenames_df$Study == "HSJ")&
                       grepl("B023", allfilenames_df$filename)] <- "HSJ23"
allfilenames_df$Patient[which(allfilenames_df$Study == "HSJ")&
                       grepl("B50", allfilenames_df$filename)] <- "HSJ50"
allfilenames_df$Patient[which(allfilenames_df$Study == "HSJ")&
                       grepl("B51-", allfilenames_df$filename)] <- "HSJ51"
allfilenames_df$Patient[which(allfilenames_df$Study == "HSJ")&
                       grepl("B051-", allfilenames_df$filename)] <- "HSJ51"
allfilenames_df$Patient[which(allfilenames_df$Study == "HSJ")&
                       grepl("B51R", allfilenames_df$filename)] <- "HSJ51R"
allfilenames_df$Patient[which(allfilenames_df$Study == "HSJ")&
                       grepl("B051R", allfilenames_df$filename)] <- "HSJ51R"
allfilenames_df$Patient[which(allfilenames_df$Study == "HSJ")&
                       grepl("B19", allfilenames_df$filename)] <- "HSJ19"
allfilenames_df$Patient[which(allfilenames_df$Study == "HSJ")&
                       grepl("B41", allfilenames_df$filename)] <- "HSJ41"
allfilenames_df$Patient[which(allfilenames_df$Study == "HSJ")&
                       grepl("B67", allfilenames_df$filename)] <- "HSJ67"
allfilenames_df$Patient[which(allfilenames_df$Study == "HSJ")&
                       grepl("B68", allfilenames_df$filename)] <- "HSJ68"
allfilenames_df$Patient[which(allfilenames_df$Study == "HSJ")&
                       grepl("B76", allfilenames_df$filename)] <- "HSJ76"
allfilenames_df$Patient[which(allfilenames_df$Study == "HSJ")&
                       grepl("B77", allfilenames_df$filename)] <- "HSJ77"

In [229]:
allfilenames_df$Patient[grepl("VM", allfilenames_df$filename)] <- "TLDC-001"
allfilenames_df$Patient[grepl("TLML-004", allfilenames_df$filename)] <- "TLML-004"
allfilenames_df$Patient[grepl("TLML-016", allfilenames_df$filename)] <- "TLML-016"
allfilenames_df$Patient[grepl("TLML_16", allfilenames_df$filename)] <- "TLML-016"
allfilenames_df$Patient[grepl("TLML_18", allfilenames_df$filename)] <- "TLML-018"

allfilenames_df$Patient[grepl("TLML-019", allfilenames_df$filename)] <- "TLML-019"
allfilenames_df$Patient[grepl("TLML-020", allfilenames_df$filename)] <- "TLML-020"
allfilenames_df$Patient[grepl("TLML_20", allfilenames_df$filename)] <- "TLML-020"

allfilenames_df$Patient[grepl("TLML-022", allfilenames_df$filename)] <- "TLML-022"
allfilenames_df$Patient[grepl("TLML_22", allfilenames_df$filename)] <- "TLML-022"

allfilenames_df$Patient[grepl("TLML-029", allfilenames_df$filename)] <- "TLML-029"
allfilenames_df$Patient[grepl("TLML_29", allfilenames_df$filename)] <- "TLML-029"


allfilenames_df$Patient[grepl("TLML_1", allfilenames_df$filename)] <- "TLML-001"
allfilenames_df$Patient[grepl("TLML_4", allfilenames_df$filename)] <- "TLML-004"
allfilenames_df$Patient[grepl("TLML_7", allfilenames_df$filename)] <- "TLML-007"


## Extract sampletype

In [202]:
allfilenames_df$Sample_type[which(allfilenames_df$Study == "NPC")] <- "Tumour"
allfilenames_df$Sample_type[which(allfilenames_df$Study == "LFS")] <- "PBMC"
allfilenames_df$Sample_type[which(allfilenames_df$Study == "INTERCEPT")] <- "PBMC"

In [203]:
allfilenames_df$Sample_type[grepl("BC", allfilenames_df$Samplename)] <- "PBMC"
allfilenames_df$Sample_type[grepl("_T_DNA", allfilenames_df$Samplename)] <- "Tumour"
allfilenames_df$Sample_type[grepl("CSF", allfilenames_df$Samplename)] <- "CSF"
allfilenames_df$Sample_type[grepl("Plasma", allfilenames_df$Samplename)] <- "Plasma"
allfilenames_df$Sample_type[grepl("Blood", allfilenames_df$Samplename)] <- "PBMC"
allfilenames_df$Sample_type[grepl("Tum", allfilenames_df$Samplename)] <- "Tumour"
allfilenames_df$Sample_type[grepl("Lymphotrack", allfilenames_df$Samplename)] <- "PBMC"

In [204]:
lines <- c('A037', 'CEM', 'Jurkat', 'L2D8', 'M36-EZM',
           'M36-TIL2-1E7P3', 'MOLT-4', 'OV7-TIL2', 'STIM1-CY3',
           'SUP-T1', 'Undetermined', 'SE14-2005', 'SE14-2033',
           'SE14-2034', 'SE14-2035')

In [205]:
allfilenames_df$Sample_type[allfilenames_df$Samplename %in% lines] <- "Lines"

In [206]:
allfilenames_df$Sample_type[grepl("TLML",allfilenames_df$Samplename) &
                            grepl("cfDNA", allfilenames_df$Samplename)] <- "Plasma"

allfilenames_df$Sample_type[grepl("TLML",allfilenames_df$Samplename) &
                            grepl("genomic", allfilenames_df$Samplename)] <- "PBMC"

allfilenames_df$Sample_type[grepl("TLDC",allfilenames_df$Samplename) &
                            grepl("cfDNA", allfilenames_df$Samplename)] <- "Plasma"

allfilenames_df$Sample_type[grepl("TLDC",allfilenames_df$Samplename) &
                            grepl("genomic", allfilenames_df$Samplename)] <- "PBMC"

In [207]:
allfilenames_df$Sample_type[grepl("INSPIRE",allfilenames_df$Samplename) &
                            grepl("ST", allfilenames_df$Samplename)] <- "Tumour"

allfilenames_df$Sample_type[grepl("INSPIRE",allfilenames_df$Samplename) &
                            grepl("C2T", allfilenames_df$Samplename)] <- "Tumour"

allfilenames_df$Sample_type[grepl("INSPIRE",allfilenames_df$Samplename) &
                          grepl("B_", allfilenames_df$Samplename) &
                          grepl("genomic", allfilenames_df$Samplename) ] <- "PBMC"

allfilenames_df$Sample_type[grepl("INSPIRE",allfilenames_df$Samplename) &
                          grepl("cfDNA", allfilenames_df$Samplename) ] <- "Plasma"

In [211]:
head(allfilenames_df)

filename,Study,Patient,Samplename,Sample_type
<chr>,<chr>,<chr>,<chr>,<chr>
IMS-001_MoST-K006_T_DNA,IMS,IMS-001,IMS-001_MoST-K006_T_DNA,Tumour
IMS-002_MoST-K011_T_DNA,IMS,IMS-002,IMS-002_MoST-K011_T_DNA,Tumour
IMS-003_MoST-K014_T_DNA,IMS,IMS-003,IMS-003_MoST-K014_T_DNA,Tumour
IMS-004_MoST-K001_T_DNA,IMS,IMS-004,IMS-004_MoST-K001_T_DNA,Tumour
IMS-005_MoST-K016_T_DNA,IMS,IMS-005,IMS-005_MoST-K016_T_DNA,Tumour
IMS-006_MoST-K017_T_DNA,IMS,IMS-006,IMS-006_MoST-K017_T_DNA,Tumour


In [230]:
write.csv(allfilenames_df, 
          file = paste0(datapath,"commonclonotypes_metadata.csv"),
          row.names = F)

In [2]:
?expand.grid

In [6]:
vec <- c("A1", "A2", "A3")
vec1 <- c("B1", "B2", "B3")

In [10]:
6*5*4*3*2

[1] 720

In [7]:
expand.grid(vec, vec1)


Var1,Var2
<fct>,<fct>
A1,B1
A2,B1
A3,B1
A1,B2
A2,B2
A3,B2
A1,B3
A2,B3
A3,B3


In [1]:
N   <- 5
vec <- c(-1, 1)
lst <- lapply(numeric(N), function(x) vec)
as.matrix(expand.grid(lst))
              

Var1,Var2,Var3,Var4,Var5
-1,-1,-1,-1,-1
1,-1,-1,-1,-1
-1,1,-1,-1,-1
1,1,-1,-1,-1
-1,-1,1,-1,-1
1,-1,1,-1,-1
-1,1,1,-1,-1
1,1,1,-1,-1
-1,-1,-1,1,-1
1,-1,-1,1,-1
